In [1]:
import os
# os.environ["CUDA_VISIBLE_DEVICES"] = "5"
import torch
import torch.nn as nn
import torchvision 
from torchvision import transforms
import numpy as np 
import tqdm

In [3]:
# train a 3 layer MLP on mnist
model = nn.Sequential(
    nn.Linear(784, 150, bias=False),
    nn.ReLU(),
    nn.Linear(150, 200, bias=False),
    nn.ReLU(),
    nn.Linear(200, 10, bias=False),
)

if torch.__version__.startswith("2"):
    print("Using torch 2.0 compile")
    torch.compile(model)
    
train_set = torchvision.datasets.MNIST(
    root='./data',
    train=True,
    download=True,
    transform=transforms.Compose([
        transforms.RandomRotation(10),
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,)),
        torch.flatten
    ])
)
test_set = torchvision.datasets.MNIST(
    root='./data',
    train=False,
    download=True,
    transform=transforms.Compose([
        transforms.
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,)),
        torch.flatten
    ])
)

train_loader = torch.utils.data.DataLoader(
    dataset=train_set,
    batch_size=128,
    shuffle=True
)
test_loader = torch.utils.data.DataLoader(
    dataset=test_set,
    batch_size=128,
    shuffle=True
)

loss_fn = nn.CrossEntropyLoss()
optim = torch.optim.SGD(model.parameters(), lr=0.1, momentum=0.9, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optim,
    mode='max',
    factor=0.1,
    patience=2,
    min_lr = 1e-4
)


epochs = 12
eval_every = 100 # batches 

best_model = None

with torch.autocast("cuda"):
    for epoch in tqdm.tqdm(range(epochs)):
        
        train_loss = 0
        eval_loss = 0
        train_acc = 0
        eval_acc = 0
        best_acc = 0
        
        # train
        for i, (x, y) in enumerate(train_loader):
            y_pred = model(x)
            loss = loss_fn(y_pred, y)
            optim.zero_grad()
            loss.backward()
            optim.step()
            
            acc = (y_pred.argmax(dim=1) == y).float().mean()
            train_acc += acc 
            train_loss += loss.item()
            
            # eval
            if i % eval_every == 0:
                for x, y in test_loader:
                    model.eval()
                    y_pred = model(x)
                    loss = loss_fn(y_pred, y)
                    
                    eval_loss += loss.item()
                    acc = (y_pred.argmax(dim=1) == y).float().mean()
                    eval_acc += acc    
                    
                print(f"Epoch {epoch}: train acc: {train_acc / eval_every}, train loss: {train_loss / eval_every},"
                    f" eval acc: {eval_acc / len(test_loader)}, eval loss: {eval_loss / len(test_loader)},"
                    f" lr: {optim.param_groups[0]['lr']}")

                
                if eval_acc / len(test_loader) > best_acc:
                    best_acc = eval_acc / len(test_loader)
                    best_model = model.state_dict()
                    
                train_loss = 0
                eval_loss = 0
                train_acc = 0
                eval_acc = 0
        
        scheduler.step(eval_acc / len(test_loader))

Using torch 2.0 compile


  0%|          | 0/12 [00:00<?, ?it/s]

Epoch 0: train acc: 0.0009374999790452421, train loss: 0.023020157814025877, eval acc: 0.13587816059589386, eval loss: 2.2883858409108995, lr: 0.1
Epoch 0: train acc: 0.8125, train loss: 0.610088667422533, eval acc: 0.9277096390724182, eval loss: 0.23557359234818928, lr: 0.1
Epoch 0: train acc: 0.9270312786102295, train loss: 0.24177031613886357, eval acc: 0.9427412748336792, eval loss: 0.18410793008117737, lr: 0.1
Epoch 0: train acc: 0.9390624761581421, train loss: 0.20490242075175047, eval acc: 0.9498615264892578, eval loss: 0.15951774462680274, lr: 0.1
Epoch 0: train acc: 0.9419531226158142, train loss: 0.1893035962432623, eval acc: 0.9521360993385315, eval loss: 0.157280340413504, lr: 0.1


  8%|▊         | 1/12 [00:21<03:56, 21.46s/it]

Epoch 1: train acc: 0.009687500074505806, train loss: 0.0009697346389293671, eval acc: 0.9570807218551636, eval loss: 0.13866330181024497, lr: 0.1
Epoch 1: train acc: 0.9591405987739563, train loss: 0.13834078039973974, eval acc: 0.9606408476829529, eval loss: 0.12890377475679676, lr: 0.1
Epoch 1: train acc: 0.9521874785423279, train loss: 0.1521232938952744, eval acc: 0.9642998576164246, eval loss: 0.12586349305472797, lr: 0.1
Epoch 1: train acc: 0.9555468559265137, train loss: 0.14240980783477425, eval acc: 0.9588607549667358, eval loss: 0.13827000522125465, lr: 0.1
Epoch 1: train acc: 0.9619531035423279, train loss: 0.12675292495638132, eval acc: 0.9672666192054749, eval loss: 0.11402801503250494, lr: 0.1


...

 83%|████████▎ | 10/12 [03:35<00:43, 21.70s/it]

Epoch 10: train acc: 0.009921874850988388, train loss: 0.00013857657089829444, eval acc: 0.9838805198669434, eval loss: 0.05268703555523217, lr: 0.00010000000000000003
Epoch 10: train acc: 0.9900000095367432, train loss: 0.030712137279333546, eval acc: 0.9838805198669434, eval loss: 0.05258933916323983, lr: 0.00010000000000000003
Epoch 10: train acc: 0.9921875, train loss: 0.024480454471195116, eval acc: 0.9824960231781006, eval loss: 0.05475099367101358, lr: 0.00010000000000000003
Epoch 10: train acc: 0.9904687404632568, train loss: 0.030018166974186897, eval acc: 0.9838805198669434, eval loss: 0.053807159809136314, lr: 0.00010000000000000003
Epoch 10: train acc: 0.9918749928474426, train loss: 0.026334327224176377, eval acc: 0.983188271522522, eval loss: 0.05370580112632317, lr: 0.00010000000000000003


 92%|█████████▏| 11/12 [03:57<00:21, 21.75s/it]

Epoch 11: train acc: 0.009921874850988388, train loss: 0.0003418070077896118, eval acc: 0.9838805198669434, eval loss: 0.05285081607466446, lr: 0.00010000000000000003
Epoch 11: train acc: 0.9900781512260437, train loss: 0.028908225242048502, eval acc: 0.9838805198669434, eval loss: 0.05257432444958323, lr: 0.00010000000000000003
Epoch 11: train acc: 0.990234375, train loss: 0.029238635851070286, eval acc: 0.9839794039726257, eval loss: 0.05254003660505802, lr: 0.00010000000000000003
Epoch 11: train acc: 0.9910937547683716, train loss: 0.029836320751346648, eval acc: 0.9832871556282043, eval loss: 0.0546753067417141, lr: 0.00010000000000000003
Epoch 11: train acc: 0.9914844036102295, train loss: 0.026752150703687222, eval acc: 0.9819027185440063, eval loss: 0.055864433333560636, lr: 0.00010000000000000003


100%|██████████| 12/12 [04:19<00:00, 21.60s/it]


In [8]:
# save the first 5000 images in the test set as numpy array
test_images = []
test_labels = []
for i, (x, y) in enumerate(test_loader):
    test_images.append(x)
    test_labels.append(y)

test_images = torch.cat(test_images).numpy()
test_labels = torch.cat(test_labels).numpy()
np.save("./data/test_images.npy", test_images)
np.save("./data/test_labels.npy", test_labels)

# save model weights as numpy array
for idx, (name, param) in enumerate(best_model.items()):
    name = name.split(".")
    name[0] = str(idx)
    name = ".".join(name)
    np.save(f"./data/linear_{name}.npy", param.detach().cpu().T.numpy()) # W @ X -> X @ W.T, cnpy reads in column-major order
    


In [10]:
param = np.load("./data/linear_0.weight.npy")
print("shape:", param.shape, ",", param.dtype, ", first 3 values (cupy): ", param[:3, 0])

shape: (784, 150) , float32 , first 3 values (cupy):  [-0.01497371 -0.0039914   0.01799681]
